In [1]:
import pandas as pd
import pandas_profiling
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from joblib import Parallel,delayed
import numpy as np
import json
import re
import time
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
import torch.nn.utils.rnn as rnn_utils
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
tqdm.pandas()

In [2]:
user_video_act_train_2_encoded_course_vec = pd.read_csv('user_video_act_train_2_encoded_course_vec.csv',converters={'label_list': eval, 'course_vecs_CNN':eval, 'course_vecs_LR':eval})

In [5]:
user_video_act_train_2_encoded_course_vec.head(2)

,activity,course_list,label_list,course_ids,video_ids,watching_counts,video_durations,local_watching_times,video_progress_times,video_start_times,video_end_times,local_start_times,local_end_times,courseListIDs,course_vecs_LR,course_vecs_CNN
0,[{'course_id': 'C_course-v1:TsinghuaX+20430064...,"['C_course-v1:TsinghuaX+20430064X+sp', 'C_cour...","[1, 1, 0, 0, 0]","[455, 492, 379, 455, 492, 492, 496, 455, 496, ...","[837, 1889, 1903, 2406, 2660, 4455, 4682, 4788...","[1, 1, 2, 1, 1, 3, 2, 1, 1, 1, 1, 1, 1, 4, 1, ...","[563.0, 166.0, 415.0, 173.0, 63.0, 262.0, 1088...","[563, 166, 415, 174, 19, 262, 2097, 254, 442, ...","[563.0, 166.0, 414.8099975585936, 173.0, 18.81...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[563.0, 166.0, 415.0, 173.0, 18.81999969482422...","['2017-07-24 13:26:56', '2017-07-06 14:47:00',...","['2017-07-24 13:36:19', '2017-07-06 14:49:46',...",[455 492 379 496 400],"[[455, -0.41622926448634734, -0.01951928858061...","[[[455, 837, -0.4624306896892794, 0.1235288815..."
1,[{'course_id': 'C_course-v1:TsinghuaX+30640014...,"['C_course-v1:TsinghuaX+30640014X+sp', 'C_cour...","[0, 1, 1, 1]","[481, 403, 404, 400, 481, 481, 403, 404, 481, ...","[3828, 4091, 4091, 5207, 5331, 6638, 8825, 882...","[33, 3, 3, 22, 2, 1, 1, 1, 1, 1, 5, 6, 16, 12,...","[748.0, 93.0, 93.0, 1176.0, 79.0, 98.0, 273.0,...","[825, 24, 24, 1429, 39, 99, 43, 43, 79, 11, 30...","[874.1998138427736, 24.5, 24.5, 2721.459894180...","[0.0, 0.0, 0.0, 19.1200008392334, 41.310001373...","[748.0, 16.229999542236328, 16.229999542236328...","['2018-03-18 13:40:31', '2019-01-31 11:54:47',...","['2018-03-18 15:14:03', '2019-01-31 14:48:29',...",[481 403 404 400],"[[481, -0.5597614352282316, -0.189907587438736...","[[[481, 3828, 2.7281347101059863, 0.6644491379..."


In [3]:
def time_transform(t):
    # 先转换为时间数组
    timeArray = time.strptime(t, "%Y-%m-%d %H:%M:%S")
    # 转换为时间戳
    timeStamp = int(time.mktime(timeArray))
    return timeStamp
def Z_score(mean_, std_,x):
    return (x-mean_)/std_
def max_mean_std(data):
    return np.max(data), np.mean(data), np.std(data)
def calculate_acc(predictions, truth):
    hit = 0
    for i in range(len(predictions)):
        if predictions[i] == truth[i]:
            hit = hit +1
    return hit/len(predictions)

In [4]:
nb_courses = 706+1
course_emb_size = 5
nb_videos = 38181+1
video_emb_size = 15
feature_size1 = course_emb_size + 42
sequence_len = 70
feature_size2 = course_emb_size + video_emb_size + 13
num_out_channel = 32
kernel_size = [3,4,5]
output_size = 32
hidden_dim = 64
num_of_lstm_layer = 1

In [5]:
class LR(nn.Module):
    
    def __init__(self):
        super(LR, self).__init__()  
        
        self.course_embedding = torch.nn.Embedding(nb_courses, course_emb_size)
        self.fc = nn.Linear(feature_size1, 1)
        self.sigmoid = nn.Sigmoid()        

    def forward(self, course_id, continues):
        #course_id  (batch_size, max_sen_len)
        #continues  (batch_size, max_sen_len, feature_size)
        emb = self.course_embedding(course_id) # (batch_size,max_sen_len, embed_size)
       
        x = torch.cat([emb,continues], 1)

        result = self.sigmoid(self.fc(x))
        
        return result

In [7]:
class MLP(nn.Module):
    
    def __init__(self):
        super(MLP, self).__init__()
        
        
        
        self.course_embedding = torch.nn.Embedding(nb_courses, course_emb_size)
        self.video_embedding = torch.nn.Embedding(nb_videos, video_emb_size)
        
        self.ReLU_activation =  nn.ReLU()
        self.tanh_activation =  nn.Tanh()
        self.sigmoid_activation = nn.Sigmoid()
        
#         self.bi_gru = nn.GRU(input_size = feature_size, hidden_size = hidden_dim, num_layers=num_of_lstm_layer,  bidirectional=False)
        
        
        self.fc1 = nn.Linear(feature_size2*sequence_len, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()
        


    def forward(self, course_id, video_id, continues,b_size):
        
        #course_id  (batch_size, max_sen_len)
        #continues  (batch_size, max_sen_len, feature_size)
        emb1 = self.course_embedding(course_id) # (batch_size,max_sen_len, embed_size)
        emb2 = self.video_embedding(video_id)
        

        x = torch.cat([emb1,emb2,continues], 2)
        
        input_x = x.view(b_size,-1)
        
        info_fusion = self.tanh_activation(self.fc1(input_x))
        info_fusion = self.tanh_activation(self.fc2(info_fusion))
        
        final_out = self.fc3(info_fusion)

        result = self.sigmoid(final_out)

        return result

In [7]:
class LSTM(nn.Module):
    
    def __init__(self):
        super(LSTM, self).__init__()
        
        
        
        self.course_embedding = torch.nn.Embedding(nb_courses, course_emb_size)
        self.video_embedding = torch.nn.Embedding(nb_videos, video_emb_size)
        
        self.ReLU_activation =  nn.ReLU()
        self.tanh_activation =  nn.Tanh()
        self.sigmoid_activation = nn.Sigmoid()
        
        
        self.bilstm = nn.LSTM(input_size = feature_size2, hidden_size = hidden_dim , num_layers=num_of_lstm_layer,  bidirectional=False)

        self.fc1 = nn.Linear(hidden_dim, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 1)
        self.sigmoid = nn.Sigmoid()
        


    def forward(self, course_id, video_id, continues):
        
        #course_id  (batch_size, max_sen_len)
        #continues  (batch_size, max_sen_len, feature_size)
        emb1 = self.course_embedding(course_id) # (batch_size,max_sen_len, embed_size)
        emb2 = self.video_embedding(video_id)

        x = torch.cat([emb1,emb2,continues], 2)
        
        x = x.permute(1, 0, 2)  #  max_sen_len * Batch_size * feature_dim
       
        bilstm_out, _ = self.bilstm(x)
        output = bilstm_out[-1]

        info_fusion = self.tanh_activation(self.fc1(output))
        info_fusion = self.tanh_activation(self.fc2(info_fusion))   
        final_out = self.fc3(info_fusion)
        result = self.sigmoid(final_out)
        

        return result

In [6]:
class BiLSTM(nn.Module):
    
    def __init__(self):
        super(BiLSTM, self).__init__()
        
        
        
        self.course_embedding = torch.nn.Embedding(nb_courses, course_emb_size)
        self.video_embedding = torch.nn.Embedding(nb_videos, video_emb_size)
        
        self.ReLU_activation =  nn.ReLU()
        self.tanh_activation =  nn.Tanh()
        self.sigmoid_activation = nn.Sigmoid()   
        
        self.bilstm = nn.LSTM(feature_size2, hidden_dim//2 , num_layers=num_of_lstm_layer,  bidirectional=True)
        
        self.fc1 = nn.Linear(hidden_dim, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, course_id, video_id, continues):
        
        #course_id  (batch_size, max_sen_len)
        #continues  (batch_size, max_sen_len, feature_size)
        emb1 = self.course_embedding(course_id) # (batch_size,max_sen_len, embed_size)
        emb2 = self.video_embedding(video_id)
        
        x = torch.cat([emb1,emb2,continues], 2)
        x = x.permute(1, 0, 2)  #  max_sen_len * Batch_size * feature_dim
       
        bilstm_out, _ = self.bilstm(x)
        output = bilstm_out[-1]
        info_fusion = self.tanh_activation(self.fc1(output))
        info_fusion = self.tanh_activation(self.fc2(info_fusion))
        final_out = self.fc3(info_fusion)
        result = self.sigmoid(final_out)

        return result

In [8]:
class GRU(nn.Module):
    
    def __init__(self):
        super(GRU, self).__init__()
        
        
        
        self.course_embedding = torch.nn.Embedding(nb_courses, course_emb_size)
        self.video_embedding = torch.nn.Embedding(nb_videos, video_emb_size)
        
        self.ReLU_activation =  nn.ReLU()
        self.tanh_activation =  nn.Tanh()
        self.sigmoid_activation = nn.Sigmoid()
        
        self.bi_gru = nn.GRU(input_size = feature_size2, hidden_size = hidden_dim, num_layers=num_of_lstm_layer,  bidirectional=False)
        
        self.fc1 = nn.Linear(hidden_dim, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 1)
        self.sigmoid = nn.Sigmoid()
        


    def forward(self, course_id, video_id, continues):
        
        #course_id  (batch_size, max_sen_len)
        #continues  (batch_size, max_sen_len, feature_size)
        emb1 = self.course_embedding(course_id) # (batch_size,max_sen_len, embed_size)
        emb2 = self.video_embedding(video_id)
        

        x = torch.cat([emb1,emb2,continues], 2)
        x = x.permute(1, 0, 2)  #  max_sen_len * Batch_size * feature_dim
       
        bigru_out, _ = self.bi_gru(x)
        output = bigru_out[-1]
        
        info_fusion = self.tanh_activation(self.fc1(output))
        
        info_fusion = self.tanh_activation(self.fc2(info_fusion))
        
        final_out = self.fc3(info_fusion)

        result = self.sigmoid(final_out)

        return result

In [27]:
class BiGRU(nn.Module):
    
    def __init__(self):
        super(BiGRU, self).__init__()
        
        self.course_embedding = torch.nn.Embedding(nb_courses, course_emb_size)
        self.video_embedding = torch.nn.Embedding(nb_videos, video_emb_size)
        
        self.ReLU_activation =  nn.ReLU()
        self.tanh_activation =  nn.Tanh()
        self.sigmoid_activation = nn.Sigmoid()
        
        self.bi_gru = nn.GRU(input_size = feature_size2, hidden_size = hidden_dim // 2 , num_layers=num_of_lstm_layer,  bidirectional=True)
        
        self.fc1 = nn.Linear(hidden_dim, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, course_id, video_id, continues):
        
        #course_id  (batch_size, max_sen_len)
        #continues  (batch_size, max_sen_len, feature_size)
        emb1 = self.course_embedding(course_id) # (batch_size,max_sen_len, embed_size)
        emb2 = self.video_embedding(video_id)

        x = torch.cat([emb1,emb2,continues], 2)
        x = x.permute(1, 0, 2)  #  max_sen_len * Batch_size * feature_dim
       
        bigru_out, _ = self.bi_gru(x)
        output = bigru_out[-1]
        
        info_fusion = self.tanh_activation(self.fc1(output))
        info_fusion = self.tanh_activation(self.fc2(info_fusion))    
        final_out = self.fc3(info_fusion)
        result = self.sigmoid(final_out)

        return result

In [30]:
class BiGRU_MultiAttent(nn.Module):
    
    def __init__(self):
        super(BiGRU_MultiAttent, self).__init__()
        
        
        
        self.course_embedding = torch.nn.Embedding(nb_courses, course_emb_size)
        self.video_embedding = torch.nn.Embedding(nb_videos, video_emb_size)
        
        self.ReLU_activation =  nn.ReLU()
        self.tanh_activation =  nn.Tanh()
        self.sigmoid_activation = nn.Sigmoid()
        self.softmax = nn.Softmax( dim=1)
        
        self.bi_gru = nn.GRU(input_size = feature_size2, hidden_size = hidden_dim , num_layers=num_of_lstm_layer,  bidirectional=True)
        
        self.self_attention = nn.MultiheadAttention(feature_size2, num_heads = 11)
        
        
        self.fc1 = nn.Linear(hidden_dim*2, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 1)
        self.sigmoid = nn.Sigmoid()

        


    def forward(self, course_id, video_id, continues):
        
        #course_id  (batch_size, max_sen_len)
        #continues  (batch_size, max_sen_len, feature_size)
        emb1 = self.course_embedding(course_id) # (batch_size,max_sen_len, embed_size)
        emb2 = self.video_embedding(video_id)
        

        x = torch.cat([emb1,emb2,continues], 2)
        x = x.permute(1, 0, 2)  #  max_sen_len * Batch_size * feature_dim
        
        #MultiHead Attention 
        attn_output, attn_output_weights = self.self_attention(x, x, x)
        
        #GRU
        bigru_out, _ = self.bi_gru(attn_output)
        output = bigru_out[-1]
        
      
        info_fusion = self.tanh_activation(self.fc1(output))
        info_fusion = self.tanh_activation(self.fc2(info_fusion))
        final_out = self.fc3(info_fusion)
        result = self.sigmoid(final_out)

        return result

In [6]:
class TextCNN(nn.Module):
    def __init__(self):
        super(TextCNN, self).__init__()
        
        self.course_embedding = torch.nn.Embedding(nb_courses, course_emb_size)
        self.video_embedding = torch.nn.Embedding(nb_videos, video_emb_size)
        
        self.ReLU_activation =  nn.ReLU()
        self.tanh_activation =  nn.Tanh()
        
        
        self.conv1 = nn.Conv1d(in_channels=feature_size2,out_channels=num_out_channel,kernel_size=kernel_size[0])
        self.maxpool1 = nn.MaxPool1d(sequence_len - kernel_size[0] + 1) 
        self.conv2 = nn.Conv1d(in_channels=feature_size2,out_channels=num_out_channel,kernel_size=kernel_size[1])
        self.maxpool2 = nn.MaxPool1d(sequence_len - kernel_size[1] + 1) 
        self.conv3 = nn.Conv1d(in_channels=feature_size2,out_channels=num_out_channel,kernel_size=kernel_size[2])
        self.maxpool3 = nn.MaxPool1d(sequence_len - kernel_size[2] + 1)
        self.fc1 = nn.Linear(num_out_channel*len(kernel_size), 64)
        self.fc2 = nn.Linear(64, 16)
        self.fc3 = nn.Linear(16, 1)
#         self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()


    def forward(self, course_id, video_id, continues):
        #course_id  (batch_size, max_sen_len)
        #continues  (batch_size, max_sen_len, feature_size)
        emb1 = self.course_embedding(course_id) # (batch_size,max_sen_len, embed_size)
        emb2 = self.video_embedding(video_id)

        x = torch.cat([emb1,emb2,continues], 2)
        x = x.permute(0, 2, 1)  # Batch_size * (feature_dim) * max_sen_len

        x1 = self.conv1(x)#.squeeze(2)  # shape = (64, num_channels, 1)(squeeze 2)
        x1 = self.ReLU_activation(x1)
        x1 = self.maxpool1(x1)
        x1 = x1.squeeze(2) 
        
        x2 = self.conv2(x)#.squeeze(2)  # shape = (64, num_channels, 1)(squeeze 2)
        x2 = self.ReLU_activation(x2)
        x2 = self.maxpool2(x2)
        x2 = x2.squeeze(2)
        
        x3 = self.conv3(x)#.squeeze(2)  # shape = (64, num_channels, 1)(squeeze 2)
        x3 = self.ReLU_activation(x3)
        x3 = self.maxpool3(x3)
        x3 = x3.squeeze(2)

        all_out = torch.cat((x1, x2, x3), dim=1)
#         print(all_out.shape)
        info_fusion = self.tanh_activation(self.fc1(all_out))       
        info_fusion = self.tanh_activation(self.fc2(info_fusion)) 
        final_out = self.fc3(info_fusion)
#         result = self.softmax(final_out)
        result = self.sigmoid(final_out)

        return result  # 返回 softmax 的结果


In [8]:
#tanh for conv results, lppool poolling used
class TextCNN_v2(nn.Module):
    def __init__(self):
        super(TextCNN_v2, self).__init__()
        
        self.course_embedding = torch.nn.Embedding(nb_courses, course_emb_size)
        self.video_embedding = torch.nn.Embedding(nb_videos, video_emb_size)
        
        self.ReLU_activation =  nn.ReLU()
        self.tanh_activation =  nn.Tanh()
        
        
        self.conv1 = nn.Conv1d(in_channels=feature_size2,out_channels=num_out_channel,kernel_size=kernel_size[0])
        self.maxpool1 = nn.MaxPool1d(sequence_len - kernel_size[0] + 1)
        self.lppool1 =  nn.LPPool1d(2, sequence_len - kernel_size[0] + 1)
        
        
        self.conv2 = nn.Conv1d(in_channels=feature_size2,out_channels=num_out_channel,kernel_size=kernel_size[1])
        self.maxpool2 = nn.MaxPool1d(sequence_len - kernel_size[1] + 1)
        self.lppool2 =  nn.LPPool1d(2, sequence_len - kernel_size[1] + 1)
        
        self.conv3 = nn.Conv1d(in_channels=feature_size2,out_channels=num_out_channel,kernel_size=kernel_size[2])
        self.maxpool3 = nn.MaxPool1d(sequence_len - kernel_size[2] + 1)
        self.lppool3 =  nn.LPPool1d(2, sequence_len - kernel_size[2] + 1)
#         self.conv4 = nn.Conv1d(in_channels=feature_size,out_channels=num_out_channel,kernel_size=kernel_size[3])
#         self.maxpool4 = nn.MaxPool1d(sequence_len - kernel_size[3] + 1)
        
        

        
        self.fc1 = nn.Linear(num_out_channel*len(kernel_size), 64)
        self.fc2 = nn.Linear(64, 16)
        self.fc3 = nn.Linear(16, 1)
#         self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()
        

    def forward(self, course_id, video_id, continues):
        #course_id  (batch_size, max_sen_len)
        #continues  (batch_size, max_sen_len, feature_size)
        emb1 = self.course_embedding(course_id) # (batch_size,max_sen_len, embed_size)
        emb2 = self.video_embedding(video_id)

        x = torch.cat([emb1,emb2,continues], 2)
#         print(x.shape)
        x = x.permute(0, 2, 1)  # Batch_size * (feature_dim) * max_sen_len
#         print('input ',x.shape)
        x1 = self.conv1(x)#.squeeze(2)  # shape = (64, num_channels, 1)(squeeze 2)
#         print('convolution result: ',x1.shape)
        x1 = self.sigmoid(x1)
#         print('actication result: ',x1.shape)
#         x1 = self.maxpool1(x1)
        x1 = self.lppool1(x1)
#         print('maxpool: ',x1.shape)
        x1 = x1.squeeze(2)
#         print('final x1: ',x1.shape)
        
        
        
        x2 = self.conv2(x)#.squeeze(2)  # shape = (64, num_channels, 1)(squeeze 2)
#         print('convolution result: ',x2.shape)
        x2 = self.sigmoid(x2)
#         print('actication result: ',x2.shape)
#         x2 = self.maxpool2(x2)
        x2 = self.lppool2(x2)
#         print('maxpool: ',x2.shape)
        x2 = x2.squeeze(2)
#         print('final x2: ',x2.shape)
        
        
        x3 = self.conv3(x)#.squeeze(2)  # shape = (64, num_channels, 1)(squeeze 2)
#         print('convolution result: ',x3.shape)
        x3 = self.sigmoid(x3)
#         print('actication result: ',x3.shape)
#         x3 = self.maxpool3(x3)
        x3 = self.lppool3(x3)
#         print('maxpool: ',x3.shape)
        x3 = x3.squeeze(2)
 
        all_out = torch.cat((x1, x2, x3), dim=1)
#         print(all_out.shape)
        info_fusion = self.tanh_activation(self.fc1(all_out))
        
        info_fusion = self.tanh_activation(self.fc2(info_fusion))
        
        final_out = self.fc3(info_fusion)
#         result = self.softmax(final_out)
        result = self.sigmoid(final_out)
        
        


        return result  # 返回 softmax 的结果


In [8]:
class TextCNN_MultiHAtten(nn.Module):
    def __init__(self):
        super(TextCNN_MultiHAtten, self).__init__()
        
        self.course_embedding = torch.nn.Embedding(nb_courses, course_emb_size)
        self.video_embedding = torch.nn.Embedding(nb_videos, video_emb_size)
        
        self.ReLU_activation =  nn.ReLU()
        self.tanh_activation =  nn.Tanh()
        
        
        self.conv1 = nn.Conv1d(in_channels=feature_size2,out_channels=num_out_channel,kernel_size=kernel_size[0])
        self.maxpool1 = nn.MaxPool1d(sequence_len - kernel_size[0] + 1)
        
        
        self.conv2 = nn.Conv1d(in_channels=feature_size2,out_channels=num_out_channel,kernel_size=kernel_size[1])
        self.maxpool2 = nn.MaxPool1d(sequence_len - kernel_size[1] + 1)
        
        self.conv3 = nn.Conv1d(in_channels=feature_size2,out_channels=num_out_channel,kernel_size=kernel_size[2])
        self.maxpool3 = nn.MaxPool1d(sequence_len - kernel_size[2] + 1)
        
#         self.conv4 = nn.Conv1d(in_channels=feature_size,out_channels=num_out_channel,kernel_size=kernel_size[3])
#         self.maxpool4 = nn.MaxPool1d(sequence_len - kernel_size[3] + 1)
        
        
        self.self_attention = nn.MultiheadAttention(num_out_channel*len(kernel_size), num_heads = 6)
        
        self.fc1 = nn.Linear(num_out_channel*len(kernel_size), 64)
        self.fc2 = nn.Linear(64, 16)
        self.fc3 = nn.Linear(16, 1)
#         self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()
        


    def forward(self, course_id, video_id, continues):
        #course_id  (batch_size, max_sen_len)
        #continues  (batch_size, max_sen_len, feature_size)
        emb1 = self.course_embedding(course_id) # (batch_size,max_sen_len, embed_size)
        emb2 = self.video_embedding(video_id)

        x = torch.cat([emb1,emb2,continues], 2)
#         print(x.shape)
        x = x.permute(0, 2, 1)  # Batch_size * (feature_dim) * max_sen_len
#         print('input ',x.shape)
        x1 = self.conv1(x)#.squeeze(2)  # shape = (64, num_channels, 1)(squeeze 2)
#         print('convolution result: ',x1.shape)
        x1 = self.ReLU_activation(x1)
#         print('actication result: ',x1.shape)
        x1 = self.maxpool1(x1)
#         print('maxpool: ',x1.shape)
        x1 = x1.squeeze(2)
#         print('final x1: ',x1.shape)
        
        
        
        x2 = self.conv2(x)#.squeeze(2)  # shape = (64, num_channels, 1)(squeeze 2)
#         print('convolution result: ',x2.shape)
        x2 = self.ReLU_activation(x2)
#         print('actication result: ',x2.shape)
        x2 = self.maxpool2(x2)
#         print('maxpool: ',x2.shape)
        x2 = x2.squeeze(2)
#         print('final x2: ',x2.shape)
        
        
        x3 = self.conv3(x)#.squeeze(2)  # shape = (64, num_channels, 1)(squeeze 2)
#         print('convolution result: ',x3.shape)
        x3 = self.ReLU_activation(x3)
#         print('actication result: ',x3.shape)
        x3 = self.maxpool3(x3)
#         print('maxpool: ',x3.shape)
        x3 = x3.squeeze(2)
#         print('final x3: ',x3.shape)

#         x4 = self.conv4(x)#.squeeze(2)  # shape = (64, num_channels, 1)(squeeze 2)
# #         print('convolution result: ',x3.shape)
#         x4 = self.ReLU_activation(x4)
# #         print('actication result: ',x3.shape)
#         x4 = self.maxpool4(x4)
# #         print('maxpool: ',x3.shape)
#         x4 = x4.squeeze(2)
        
        
#         x2 = self.conv2(x)#.squeeze(2)
#         x3 = self.conv3(x)#.squeeze(2)
        all_out = torch.cat((x1, x2, x3), dim=1)
#         print('all_out: ',all_out.shape)
        
        all_out = all_out.unsqueeze(2)
#         print('all_out: ',all_out.shape)
        all_out = all_out.permute(2, 0, 1) 
#         print('all_out: ',all_out.shape)
        
        attn_output, attn_output_weights = self.self_attention(all_out, all_out, all_out)
#         print('attn_output: ',attn_output.shape)
        attn_output = attn_output.permute(1, 2, 0)
#         print('attn_output2: ',attn_output.shape)
        
        attn_output = attn_output.squeeze(2)
#         print('attn_output3: ',attn_output.shape)
        info_fusion = self.tanh_activation(self.fc1(attn_output))
        
        info_fusion = self.tanh_activation(self.fc2(info_fusion))
        
        final_out = self.fc3(info_fusion)
#         result = self.softmax(final_out)
        result = self.sigmoid(final_out)
        
        


        return result  # 返回 softmax 的结果


In [38]:
kernel_size = [2,3,4,5]
class TextCNN_4conv(nn.Module):
    def __init__(self):
        super(TextCNN_4conv, self).__init__()
        
        self.course_embedding = torch.nn.Embedding(nb_courses, course_emb_size)
        self.video_embedding = torch.nn.Embedding(nb_videos, video_emb_size)
        
        self.ReLU_activation =  nn.ReLU()
        self.tanh_activation =  nn.Tanh()
        
        
        self.conv1 = nn.Conv1d(in_channels=feature_size2,out_channels=num_out_channel,kernel_size=kernel_size[0])
        self.maxpool1 = nn.MaxPool1d(sequence_len - kernel_size[0] + 1)
        
        self.conv2 = nn.Conv1d(in_channels=feature_size2,out_channels=num_out_channel,kernel_size=kernel_size[1])
        self.maxpool2 = nn.MaxPool1d(sequence_len - kernel_size[1] + 1)
        
        self.conv3 = nn.Conv1d(in_channels=feature_size2,out_channels=num_out_channel,kernel_size=kernel_size[2])
        self.maxpool3 = nn.MaxPool1d(sequence_len - kernel_size[2] + 1)
        
        self.conv4 = nn.Conv1d(in_channels=feature_size2,out_channels=num_out_channel,kernel_size=kernel_size[3])
        self.maxpool4 = nn.MaxPool1d(sequence_len - kernel_size[3] + 1)
        
        self.fc1 = nn.Linear(num_out_channel*len(kernel_size), 64)
        self.fc2 = nn.Linear(64, 16)
        self.fc3 = nn.Linear(16, 1)
#         self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()
        

    def forward(self, course_id, video_id, continues):
        #course_id  (batch_size, max_sen_len)
        #continues  (batch_size, max_sen_len, feature_size)
        emb1 = self.course_embedding(course_id) # (batch_size,max_sen_len, embed_size)
        emb2 = self.video_embedding(video_id)

        x = torch.cat([emb1,emb2,continues], 2)
#         print(x.shape)
        x = x.permute(0, 2, 1)  # Batch_size * (feature_dim) * max_sen_len
#         print('input ',x.shape)
        x1 = self.conv1(x)#.squeeze(2)  # shape = (64, num_channels, 1)(squeeze 2)
#         print('convolution result: ',x1.shape)
        x1 = self.ReLU_activation(x1)
        x1 = self.maxpool1(x1)
        x1 = x1.squeeze(2)
        
        
        
        x2 = self.conv2(x)#.squeeze(2)  # shape = (64, num_channels, 1)(squeeze 2)
#         print('convolution result: ',x2.shape)
        x2 = self.ReLU_activation(x2)
#         print('actication result: ',x2.shape)
        x2 = self.maxpool2(x2)
#         print('maxpool: ',x2.shape)
        x2 = x2.squeeze(2)
#         print('final x2: ',x2.shape)
        
        
        x3 = self.conv3(x)#.squeeze(2)  # shape = (64, num_channels, 1)(squeeze 2)
#         print('convolution result: ',x3.shape)
        x3 = self.ReLU_activation(x3)
#         print('actication result: ',x3.shape)
        x3 = self.maxpool3(x3)
#         print('maxpool: ',x3.shape)
        x3 = x3.squeeze(2)
#         print('final x3: ',x3.shape)

        x4 = self.conv4(x)#.squeeze(2)  # shape = (64, num_channels, 1)(squeeze 2)
#         print('convolution result: ',x3.shape)
        x4 = self.ReLU_activation(x4)
#         print('actication result: ',x3.shape)
        x4 = self.maxpool4(x4)
#         print('maxpool: ',x3.shape)
        x4 = x4.squeeze(2)
        
        
#         x2 = self.conv2(x)#.squeeze(2)
#         x3 = self.conv3(x)#.squeeze(2)
        all_out = torch.cat((x1, x2, x3,x4), dim=1)
#         print(all_out.shape)
        info_fusion = self.tanh_activation(self.fc1(all_out))
        
        info_fusion = self.tanh_activation(self.fc2(info_fusion))
        
        final_out = self.fc3(info_fusion)
#         result = self.softmax(final_out)
        result = self.sigmoid(final_out)
        
        


        return result  # 返回 softmax 的结果

In [7]:
class LR_CNN(nn.Module):
    
    def __init__(self):
        super(LR_CNN, self).__init__()  
        
        self.course_embedding = torch.nn.Embedding(nb_courses, course_emb_size)
        self.video_embedding = torch.nn.Embedding(nb_videos, video_emb_size)
        
                
        self.conv1 = nn.Conv1d(in_channels=feature_size2,out_channels=num_out_channel,kernel_size=kernel_size[0])
        self.maxpool1 = nn.MaxPool1d(sequence_len - kernel_size[0] + 1)
        
        
        self.conv2 = nn.Conv1d(in_channels=feature_size2,out_channels=num_out_channel,kernel_size=kernel_size[1])
        self.maxpool2 = nn.MaxPool1d(sequence_len - kernel_size[1] + 1)
        
        self.conv3 = nn.Conv1d(in_channels=feature_size2,out_channels=num_out_channel,kernel_size=kernel_size[2])
        self.maxpool3 = nn.MaxPool1d(sequence_len - kernel_size[2] + 1)
    
        
        self.lr_fc = nn.Linear(feature_size1, 1)
        
        self.fc1 = nn.Linear(num_out_channel*len(kernel_size), 64)
        self.fc2 = nn.Linear(64, 16)
        self.fc3 = nn.Linear(16, 1)
        
        self.ReLU_activation =  nn.ReLU()
        self.tanh_activation =  nn.Tanh()
        self.sigmoid_activation = nn.Sigmoid()   
        
        self.final_fc = nn.Linear(2, 1)

    def forward(self, courseid_LR,courseid_CNN,continuesfeature1,continuesfeature2,videoid):
        
        #course_id  (batch_size, max_sen_len)
        #continues  (batch_size, max_sen_len, feature_size)
        emb1_LR = self.course_embedding(courseid_LR)
        emb1_CNN = self.course_embedding(courseid_CNN)# (batch_size,max_sen_len, embed_size)
        emb2 = self.video_embedding(videoid)
        
        # LR part
        LR_x = torch.cat([emb1_LR,continuesfeature1], 1)
        LR_result = self.sigmoid_activation(self.lr_fc(LR_x))
        
        #CNN part

        CNN_x = torch.cat([emb1_CNN,emb2,continuesfeature2], 2)
        CNN_x = CNN_x.permute(0, 2, 1)  # Batch_size * (feature_dim) * max_sen_len
        x1 = self.conv1(CNN_x)#.squeeze(2)  # shape = (64, num_channels, 1)(squeeze 2)
        x1 = self.ReLU_activation(x1)
        x1 = self.maxpool1(x1)
        x1 = x1.squeeze(2)
#         print('final x1: ',x1.shape)
       
        x2 = self.conv2(CNN_x)#.squeeze(2)  # shape = (64, num_channels, 1)(squeeze 2)
        x2 = self.ReLU_activation(x2)
        x2 = self.maxpool2(x2)
        x2 = x2.squeeze(2)
        
        x3 = self.conv3(CNN_x)#.squeeze(2)  # shape = (64, num_channels, 1)(squeeze 2)
        x3 = self.ReLU_activation(x3)
        x3 = self.maxpool3(x3)
        x3 = x3.squeeze(2)
     
        
        all_out = torch.cat((x1, x2, x3), dim=1)
        info_fusion = self.tanh_activation(self.fc1(all_out))
        info_fusion = self.tanh_activation(self.fc2(info_fusion)) 
        final_out = self.fc3(info_fusion)
        CNN_result = self.sigmoid_activation(final_out)
        
        #combine two result
        final_input = torch.cat((LR_result, CNN_result), dim=1)
#         print(final_input.shape)
        result = self.sigmoid_activation(self.final_fc(final_input))        
        
        return result,LR_result,CNN_result

In [27]:
class LR_CNN_atten(nn.Module):
    
    def __init__(self):
        super(LR_CNN_atten, self).__init__()  
        
        self.course_embedding = torch.nn.Embedding(nb_courses, course_emb_size)
        self.video_embedding = torch.nn.Embedding(nb_videos, video_emb_size)
        
                
        self.conv1 = nn.Conv1d(in_channels=feature_size2,out_channels=num_out_channel,kernel_size=kernel_size[0])
        self.maxpool1 = nn.MaxPool1d(sequence_len - kernel_size[0] + 1)
        
        
        self.conv2 = nn.Conv1d(in_channels=feature_size2,out_channels=num_out_channel,kernel_size=kernel_size[1])
        self.maxpool2 = nn.MaxPool1d(sequence_len - kernel_size[1] + 1)
        
        self.conv3 = nn.Conv1d(in_channels=feature_size2,out_channels=num_out_channel,kernel_size=kernel_size[2])
        self.maxpool3 = nn.MaxPool1d(sequence_len - kernel_size[2] + 1)
        
#         self.conv4 = nn.Conv1d(in_channels=feature_size,out_channels=num_out_channel,kernel_size=kernel_size[3])
#         self.maxpool4 = nn.MaxPool1d(sequence_len - kernel_size[3] + 1)
        self.self_attention = nn.MultiheadAttention(num_out_channel*len(kernel_size), num_heads = 6)
        self.lr_fc = nn.Linear(feature_size1, 1)
        
        self.fc1 = nn.Linear(num_out_channel*len(kernel_size), 64)
        self.fc2 = nn.Linear(64, 16)
        self.fc3 = nn.Linear(16, 1)
        
        self.ReLU_activation =  nn.ReLU()
        self.tanh_activation =  nn.Tanh()
        self.sigmoid_activation = nn.Sigmoid()   
        
        self.final_fc = nn.Linear(2, 1)

    def forward(self, courseid_LR,courseid_CNN,continuesfeature1,continuesfeature2,videoid):
        
        #course_id  (batch_size, max_sen_len)
        #continues  (batch_size, max_sen_len, feature_size)
        emb1_LR = self.course_embedding(courseid_LR)
        emb1_CNN = self.course_embedding(courseid_CNN)# (batch_size,max_sen_len, embed_size)
        emb2 = self.video_embedding(videoid)
        
        # LR part
        LR_x = torch.cat([emb1_LR,continuesfeature1], 1)
        LR_result = self.sigmoid_activation(self.lr_fc(LR_x))
        
        #CNN part
#         print('emb1_CNN:',emb1_CNN.shape)
#         print('emb2:',emb2.shape)
#         print('continuesfeature2:',continuesfeature2.shape)
        
        CNN_x = torch.cat([emb1_CNN,emb2,continuesfeature2], 2)
        CNN_x = CNN_x.permute(0, 2, 1)  # Batch_size * (feature_dim) * max_sen_len
        x1 = self.conv1(CNN_x)#.squeeze(2)  # shape = (64, num_channels, 1)(squeeze 2)
        x1 = self.ReLU_activation(x1)
        x1 = self.maxpool1(x1)
        x1 = x1.squeeze(2)
#         print('final x1: ',x1.shape)
       
        x2 = self.conv2(CNN_x)#.squeeze(2)  # shape = (64, num_channels, 1)(squeeze 2)
        x2 = self.ReLU_activation(x2)
        x2 = self.maxpool2(x2)
        x2 = x2.squeeze(2)
        
        x3 = self.conv3(CNN_x)#.squeeze(2)  # shape = (64, num_channels, 1)(squeeze 2)
        x3 = self.ReLU_activation(x3)
        x3 = self.maxpool3(x3)
        x3 = x3.squeeze(2)

#         x4 = self.conv4(CNN_x)#.squeeze(2)  # shape = (64, num_channels, 1)(squeeze 2)
#         x4 = self.ReLU_activation(x4)
#         x4 = self.maxpool4(x4)
#         x4 = x4.squeeze(2)        
        
        all_out = torch.cat((x1, x2, x3), dim=1)
        
        all_out = all_out.unsqueeze(2)
#         print('all_out: ',all_out.shape)
        all_out = all_out.permute(2, 0, 1) 
#         print('all_out: ',all_out.shape)
        
        attn_output, attn_output_weights = self.self_attention(all_out, all_out, all_out)
#         print('attn_output: ',attn_output.shape)
        attn_output = attn_output.permute(1, 2, 0)
#         print('attn_output2: ',attn_output.shape)
        
        attn_output = attn_output.squeeze(2)
        
        
        info_fusion = self.tanh_activation(self.fc1(attn_output))
        info_fusion = self.tanh_activation(self.fc2(info_fusion)) 
        final_out = self.fc3(info_fusion)
        CNN_result = self.sigmoid_activation(final_out)
        
        
        #combine two result
        final_input = torch.cat((LR_result, CNN_result), dim=1)
#         print(final_input.shape)
        result = self.sigmoid_activation(self.final_fc(final_input))        
        
        return result,LR_result,CNN_result

In [55]:
class LR_BiGRU(nn.Module):
    
    def __init__(self):
        super(LR_BiGRU, self).__init__()  
        
        self.course_embedding = torch.nn.Embedding(nb_courses, course_emb_size)
        self.video_embedding = torch.nn.Embedding(nb_videos, video_emb_size)
        
                
        self.bi_gru = nn.GRU(input_size = feature_size2, hidden_size = hidden_dim//2 , num_layers=num_of_lstm_layer,  bidirectional=True)
        
        
        self.lr_fc = nn.Linear(feature_size1, 1)
        
        self.fc1 = nn.Linear(hidden_dim, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 1)
        
        self.ReLU_activation =  nn.ReLU()
        self.tanh_activation =  nn.Tanh()
        self.sigmoid_activation = nn.Sigmoid()   
        
        self.final_fc = nn.Linear(2, 1)

    def forward(self, courseid_LR,courseid_CNN,continuesfeature1,continuesfeature2,videoid):
        
        #course_id  (batch_size, max_sen_len)
        #continues  (batch_size, max_sen_len, feature_size)
        emb1_LR = self.course_embedding(courseid_LR)
        emb1_CNN = self.course_embedding(courseid_CNN)# (batch_size,max_sen_len, embed_size)
        emb2 = self.video_embedding(videoid)
        
        # LR part
        LR_x = torch.cat([emb1_LR,continuesfeature1], 1)
        LR_result = self.sigmoid_activation(self.lr_fc(LR_x))
        
        #GRU part
        
        GRU_x = torch.cat([emb1_CNN,emb2,continuesfeature2], 2)
        GRU_x = GRU_x.permute(1, 0, 2)  # Batch_size * (feature_dim) * max_sen_len
        bigru_out, _ = self.bi_gru(GRU_x)        
        output = bigru_out[-1]      
        
        info_fusion = self.tanh_activation(self.fc1(output))
        info_fusion = self.tanh_activation(self.fc2(info_fusion)) 
        final_out = self.fc3(info_fusion)
        GRU_result = self.sigmoid_activation(final_out)
        
        
        #combine two result
        final_input = torch.cat((LR_result, GRU_result), dim=1)
        result = self.sigmoid_activation(self.final_fc(final_input))        
        
        return result,LR_result,GRU_result

In [22]:
class LR_single_direc_GRU(nn.Module):
    
    def __init__(self):
        super(LR_single_direc_GRU, self).__init__()  
        
        self.course_embedding = torch.nn.Embedding(nb_courses, course_emb_size)
        self.video_embedding = torch.nn.Embedding(nb_videos, video_emb_size)
        
                
        self.bi_gru = nn.GRU(input_size = feature_size2, hidden_size = hidden_dim , num_layers=num_of_lstm_layer,  bidirectional=False)
        
        
        self.lr_fc = nn.Linear(feature_size1, 1)
        
        self.fc1 = nn.Linear(hidden_dim, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 1)
        
        self.ReLU_activation =  nn.ReLU()
        self.tanh_activation =  nn.Tanh()
        self.sigmoid_activation = nn.Sigmoid()   
        
        self.final_fc = nn.Linear(2, 1)

    def forward(self, courseid_LR,courseid_CNN,continuesfeature1,continuesfeature2,videoid):
        
        #course_id  (batch_size, max_sen_len)
        #continues  (batch_size, max_sen_len, feature_size)
        emb1_LR = self.course_embedding(courseid_LR)
        emb1_CNN = self.course_embedding(courseid_CNN)# (batch_size,max_sen_len, embed_size)
        emb2 = self.video_embedding(videoid)
        
        # LR part
        LR_x = torch.cat([emb1_LR,continuesfeature1], 1)
        LR_result = self.sigmoid_activation(self.lr_fc(LR_x))
        
        #GRU part
        
        GRU_x = torch.cat([emb1_CNN,emb2,continuesfeature2], 2)
        GRU_x = GRU_x.permute(1, 0, 2)  # Batch_size * (feature_dim) * max_sen_len
        bigru_out, _ = self.bi_gru(GRU_x)        
        output = bigru_out[-1]      
        
        info_fusion = self.tanh_activation(self.fc1(output))
        info_fusion = self.tanh_activation(self.fc2(info_fusion)) 
        final_out = self.fc3(info_fusion)
        GRU_result = self.sigmoid_activation(final_out)
        
        #combine two result
        final_input = torch.cat((LR_result, GRU_result), dim=1)
        result = self.sigmoid_activation(self.final_fc(final_input))        
        
        return result,LR_result,GRU_result

In [16]:
class LR_BiLSTM(nn.Module):
    
    def __init__(self):
        super(LR_BiLSTM, self).__init__()  
        
        self.course_embedding = torch.nn.Embedding(nb_courses, course_emb_size)
        self.video_embedding = torch.nn.Embedding(nb_videos, video_emb_size)
        
                
        self.bilstm = nn.LSTM(input_size = feature_size2, hidden_size = hidden_dim , num_layers=num_of_lstm_layer,  bidirectional=True)
        
        
        self.lr_fc = nn.Linear(feature_size1, 1)
        
        self.fc1 = nn.Linear(hidden_dim*2, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 1)
        
        self.ReLU_activation =  nn.ReLU()
        self.tanh_activation =  nn.Tanh()
        self.sigmoid_activation = nn.Sigmoid()   
        
        self.final_fc = nn.Linear(2, 1)

    def forward(self, courseid_LR,courseid_CNN,continuesfeature1,continuesfeature2,videoid):
        
        #course_id  (batch_size, max_sen_len)
        #continues  (batch_size, max_sen_len, feature_size)
        emb1_LR = self.course_embedding(courseid_LR)
        emb1_CNN = self.course_embedding(courseid_CNN)# (batch_size,max_sen_len, embed_size)
        emb2 = self.video_embedding(videoid)
        
        # LR part
        LR_x = torch.cat([emb1_LR,continuesfeature1], 1)
        LR_result = self.sigmoid_activation(self.lr_fc(LR_x))
        
        #GRU part
        
        LSTM_x = torch.cat([emb1_CNN,emb2,continuesfeature2], 2)
        LSTM_x = LSTM_x.permute(1, 0, 2)  # Batch_size * (feature_dim) * max_sen_len
        bilstm_out, _ = self.bilstm(LSTM_x)        
        output = bilstm_out[-1]      
        
        info_fusion = self.tanh_activation(self.fc1(output))
        info_fusion = self.tanh_activation(self.fc2(info_fusion)) 
        final_out = self.fc3(info_fusion)
        LSTM_result = self.sigmoid_activation(final_out)
        
        
        #combine two result
        final_input = torch.cat((LR_result, LSTM_result), dim=1)
        result = self.sigmoid_activation(self.final_fc(final_input))        
        
        return result,LR_result,LSTM_result

In [17]:
class LR_single_direc_LSTM(nn.Module):
    
    def __init__(self):
        super(LR_single_direc_LSTM, self).__init__()  
        
        self.course_embedding = torch.nn.Embedding(nb_courses, course_emb_size)
        self.video_embedding = torch.nn.Embedding(nb_videos, video_emb_size)
        
                
        self.bilstm = nn.LSTM(input_size = feature_size2, hidden_size = hidden_dim , num_layers=num_of_lstm_layer,  bidirectional=False)
        
        
        self.lr_fc = nn.Linear(feature_size1, 1)
        
        self.fc1 = nn.Linear(hidden_dim, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 1)
        
        self.ReLU_activation =  nn.ReLU()
        self.tanh_activation =  nn.Tanh()
        self.sigmoid_activation = nn.Sigmoid()   
        
        self.final_fc = nn.Linear(2, 1)

    def forward(self, courseid_LR,courseid_CNN,continuesfeature1,continuesfeature2,videoid):
        
        #course_id  (batch_size, max_sen_len)
        #continues  (batch_size, max_sen_len, feature_size)
        emb1_LR = self.course_embedding(courseid_LR)
        emb1_CNN = self.course_embedding(courseid_CNN)# (batch_size,max_sen_len, embed_size)
        emb2 = self.video_embedding(videoid)
        
        # LR part
        LR_x = torch.cat([emb1_LR,continuesfeature1], 1)
        LR_result = self.sigmoid_activation(self.lr_fc(LR_x))
        
        #GRU part
        
        LSTM_x = torch.cat([emb1_CNN,emb2,continuesfeature2], 2)
        LSTM_x = LSTM_x.permute(1, 0, 2)  # Batch_size * (feature_dim) * max_sen_len
        bilstm_out, _ = self.bilstm(LSTM_x)        
        output = bilstm_out[-1]      
        
        info_fusion = self.tanh_activation(self.fc1(output))
        info_fusion = self.tanh_activation(self.fc2(info_fusion)) 
        final_out = self.fc3(info_fusion)
        LSTM_result = self.sigmoid_activation(final_out)
        
        
        #combine two result
        final_input = torch.cat((LR_result, LSTM_result), dim=1)
        result = self.sigmoid_activation(self.final_fc(final_input))        
        
        return result,LR_result,LSTM_result

In [18]:
class LR_MLP(nn.Module):
    
    def __init__(self):
        super(LR_MLP, self).__init__()
        
        
        
        self.course_embedding = torch.nn.Embedding(nb_courses, course_emb_size)
        self.video_embedding = torch.nn.Embedding(nb_videos, video_emb_size)
        
        self.ReLU_activation =  nn.ReLU()
        self.tanh_activation =  nn.Tanh()
        self.sigmoid_activation = nn.Sigmoid()
        
        self.lr_fc = nn.Linear(feature_size1, 1)
        
        self.fc1 = nn.Linear(feature_size2*sequence_len, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)
#         self.sigmoid = nn.Sigmoid()
        self.final_fc = nn.Linear(2, 1)
        


    def forward(self, courseid_LR,courseid_CNN,continuesfeature1,continuesfeature2,videoid,b_size):
        
        #course_id  (batch_size, max_sen_len)
        #continues  (batch_size, max_sen_len, feature_size)
        emb1_LR = self.course_embedding(courseid_LR)
        emb1_CNN = self.course_embedding(courseid_CNN)# (batch_size,max_sen_len, embed_size)
        emb2 = self.video_embedding(videoid)
        
        # LR part
        LR_x = torch.cat([emb1_LR,continuesfeature1], 1)
        LR_result = self.sigmoid_activation(self.lr_fc(LR_x))
        
        
        #MLP part
        x = torch.cat([emb1_CNN,emb2,continuesfeature2], 2)
        
        input_x = x.view(b_size,-1)   
        info_fusion = self.tanh_activation(self.fc1(input_x))
        info_fusion = self.tanh_activation(self.fc2(info_fusion))
        final_out = self.fc3(info_fusion)
        MLP_result = self.sigmoid_activation(final_out)
        
        #combine two results
        final_input = torch.cat((LR_result, MLP_result), dim=1)
        result = self.sigmoid_activation(self.final_fc(final_input))      

        return  result,LR_result,MLP_result

In [28]:
# model_MLP = MLP()
# model_MLP.load_state_dict(torch.load('mlp_25epoch.model'))
# model_MLP.eval()

# model_GRU = GRU()
# model_GRU.load_state_dict(torch.load('single_direc_gru_25epoch.model'))
# model_GRU.eval()

model_BiGRU = BiGRU()
model_BiGRU.load_state_dict(torch.load('gru_25epoch.model'))
model_BiGRU.eval()

# # model_BiGRU_MultiAttent = BiGRU_MultiAttent()
# # model_BiGRU_MultiAttent.load_state_dict(torch.load('gru_MultiHeadAtten_30epoch.model'))
# # model_BiGRU_MultiAttent.eval()

model_LSTM = LSTM()
model_LSTM.load_state_dict(torch.load('single_direc_lstm_25epoch.model'))
model_LSTM.eval()

model_BiLSTM = BiLSTM()
model_BiLSTM.load_state_dict(torch.load('Bilstm_25epoch.model'))
model_BiLSTM.eval()

model_LR = LR()
model_LR.load_state_dict(torch.load('lr_40epoch.model'))
model_LR.eval()

# model_TextCNN = TextCNN()
# model_TextCNN.load_state_dict(torch.load('textcnn_25epoch.model'))
# model_TextCNN.eval()

LR(
  (course_embedding): Embedding(707, 5)
  (fc): Linear(in_features=47, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [17]:
def prediction_single_model(course_vecs_LR,course_vecs_CNN):
    course_id = []
    video_id = []
    continues_feature = []
     
    
    #get x for LR
    course_info_LR = course_vecs_LR

    course_id_LR = []
    continues_feature1 = []
    for i in range(len(course_info_LR)): #get a course
        c = course_info_LR[i]
        cat_feture1 = c[0]       #get course_id
        con_feture = c[1:]        #get continues features
        course_id_LR.append(cat_feture1) 
        continues_feature1.append(con_feture)
        
        
    #get x for CNN
    course_list = course_vecs_CNN
    course_id_CNN = []
    video_id = []
    continues_feature2 = []

    for i in range(len(course_list)): #get a course 
        c = course_list[i]
        course_cat1 = []
        course_cat2 = []
        course_con = []
        for j in range(len(c)):       #get a subject
            s = c[j]
            cat_feture1 = s[0]       #get course_id and video_id
            cat_feture2 = s[1]
            course_cat1.append(cat_feture1)
            course_cat2.append(cat_feture2)
            con_feture = s[2:]        #get continues features
            course_con.append(con_feture)
        if len(course_cat1)<sequence_len:
            length = sequence_len - len(course_cat1)
            temp_course_id = [706] * length
            temp_video_id = [38180] * length
            temp2 = [[0,0,0,0,0,0,0,0,0,0,0,0,0]] * length
            course_cat1 = course_cat1 + temp_course_id
            course_cat2 = course_cat2 + temp_video_id
            course_con = course_con + temp2

        course_id_CNN.append(course_cat1) 
        video_id.append(course_cat2) 
        continues_feature2.append(course_con)

    # to tensor
    continues_feature1 = torch.tensor(continues_feature1)
    course_id_LR = torch.tensor(course_id_LR).clone().long()
    
    
    continues_feature2 = torch.tensor(continues_feature2)
    course_id_CNN = torch.tensor(course_id_CNN).clone().long()
    video_id = torch.tensor(video_id).clone().long()
    
    
    predictions_gru = model_LSTM(course_id_CNN,video_id,continues_feature2)
#     predictions_gru = model_MLP(course_id_CNN,video_id,continues_feature2,len(course_id_LR))

#     predictions_lr = model_LR(course_id_LR,continues_feature1)
    
    predictions = torch.flatten(predictions_gru)
    predictions_round = torch.round(predictions)
    results = predictions_round.detach().numpy().tolist()
    results = list(map(int,results))
    
    results_prob = predictions.detach().numpy().tolist()
#     result_prob = list(map(int,result_prob))
    

    return results, results_prob

In [29]:
def prediction_train_seperately(course_vecs_LR,course_vecs_CNN):
    course_id = []
    video_id = []
    continues_feature = []
     
    
    #get x for LR
    course_info_LR = course_vecs_LR

    course_id_LR = []
    continues_feature1 = []
    for i in range(len(course_info_LR)): #get a course
        c = course_info_LR[i]
        cat_feture1 = c[0]       #get course_id
        con_feture = c[1:]        #get continues features
        course_id_LR.append(cat_feture1) 
        continues_feature1.append(con_feture)
        
        
    #get x for CNN
    course_list = course_vecs_CNN
    course_id_CNN = []
    video_id = []
    continues_feature2 = []

    for i in range(len(course_list)): #get a course 
        c = course_list[i]
        course_cat1 = []
        course_cat2 = []
        course_con = []
        for j in range(len(c)):       #get a subject
            s = c[j]
            cat_feture1 = s[0]       #get course_id and video_id
            cat_feture2 = s[1]
            course_cat1.append(cat_feture1)
            course_cat2.append(cat_feture2)
            con_feture = s[2:]        #get continues features
            course_con.append(con_feture)
        if len(course_cat1)<sequence_len:
            length = sequence_len - len(course_cat1)
            temp_course_id = [706] * length
            temp_video_id = [38180] * length
            temp2 = [[0,0,0,0,0,0,0,0,0,0,0,0,0]] * length
            course_cat1 = course_cat1 + temp_course_id
            course_cat2 = course_cat2 + temp_video_id
            course_con = course_con + temp2

        course_id_CNN.append(course_cat1) 
        video_id.append(course_cat2) 
        continues_feature2.append(course_con)

    # to tensor
    continues_feature1 = torch.tensor(continues_feature1)
    course_id_LR = torch.tensor(course_id_LR).clone().long()
    
    
    continues_feature2 = torch.tensor(continues_feature2)
    course_id_CNN = torch.tensor(course_id_CNN).clone().long()
    video_id = torch.tensor(video_id).clone().long()
    
    
    predictions_gru = model_BiGRU(course_id_CNN,video_id,continues_feature2)
#     predictions_gru = model_MLP(course_id_CNN,video_id,continues_feature2,len(course_id_LR))
    predictions_lr = model_LR(course_id_LR,continues_feature1)
    predictions = torch.flatten((predictions_gru+predictions_lr)/2)
    predictions_round = torch.round(predictions)
    results = predictions_round.detach().numpy().tolist()
    results = list(map(int,results))
    
    results_prob = predictions.detach().numpy().tolist()
#     result_prob = list(map(int,result_prob))
    

    return results, results_prob

In [30]:
def prediction_train_jointly(course_vecs_LR,course_vecs_CNN):
    course_id = []
    video_id = []
    continues_feature = []
     
    
    #get x for LR
    course_info_LR = course_vecs_LR

    course_id_LR = []
    continues_feature1 = []
    for i in range(len(course_info_LR)): #get a course
        c = course_info_LR[i]
        cat_feture1 = c[0]       #get course_id
        con_feture = c[1:]        #get continues features
        course_id_LR.append(cat_feture1) 
        continues_feature1.append(con_feture)
        
        
    #get x for CNN
    course_list = course_vecs_CNN
    course_id_CNN = []
    video_id = []
    continues_feature2 = []

    for i in range(len(course_list)): #get a course 
        c = course_list[i]
        course_cat1 = []
        course_cat2 = []
        course_con = []
        for j in range(len(c)):       #get a subject
            s = c[j]
            cat_feture1 = s[0]       #get course_id and video_id
            cat_feture2 = s[1]
            course_cat1.append(cat_feture1)
            course_cat2.append(cat_feture2)
            con_feture = s[2:]        #get continues features
            course_con.append(con_feture)
        if len(course_cat1)<sequence_len:
            length = sequence_len - len(course_cat1)
            temp_course_id = [706] * length
            temp_video_id = [38180] * length
            temp2 = [[0,0,0,0,0,0,0,0,0,0,0,0,0]] * length
            course_cat1 = course_cat1 + temp_course_id
            course_cat2 = course_cat2 + temp_video_id
            course_con = course_con + temp2

        course_id_CNN.append(course_cat1) 
        video_id.append(course_cat2) 
        continues_feature2.append(course_con)

    # to tensor
    continues_feature1 = torch.tensor(continues_feature1)
    course_id_LR = torch.tensor(course_id_LR).clone().long()
    
    
    continues_feature2 = torch.tensor(continues_feature2)
    course_id_CNN = torch.tensor(course_id_CNN).clone().long()
    video_id = torch.tensor(video_id).clone().long()
    
    
    predictions,_,_ = model_LR_CNN_atten(course_id_LR,course_id_CNN,continues_feature1,continues_feature2,video_id)
#     predictions,_,_ = model_LR_MLP(course_id_LR,course_id_CNN,continues_feature1,continues_feature2,video_id,len(course_id_LR))
    predictions_round = torch.round(torch.flatten(predictions))
    results = predictions_round.detach().numpy().tolist()
    results = list(map(int,results))
    results_prob = torch.flatten(predictions).detach().numpy().tolist()
#     predictions,_,_ = model(course_id_LR,course_id_CNN,continues_feature1,continues_feature2,video_id)
#     predictions = torch.round(torch.flatten(predictions))
#     results = predictions.detach().numpy().tolist()
#     results = list(map(int,results))
    return results, results_prob

In [79]:
user_video_act_train_2_encoded_course_vec['predictions'] = user_video_act_train_2_encoded_course_vec.progress_apply(lambda row: prediction_train_seperately(row['course_vecs_LR'],row['course_vecs_CNN']), axis=1)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7296/7296 [02:02<00:00, 59.50it/s]


In [20]:
# final_result = user_video_act_train_2_encoded_course_vec[['predictions']].values.tolist()
# ground_truth = user_video_act_train_2_encoded_course_vec[['label_list']].values.tolist()
# # test = final_result.values.tolist()
# final_result = [ item for elem in final_result for item in elem]
# final_result = [ item for elem in final_result for item in elem]
# ground_truth = [ item for elem in ground_truth for item in elem]
# ground_truth = [ item for elem in ground_truth for item in elem]
# acc = calculate_acc(final_result,ground_truth)
# acc

0.8284921550813956

In [30]:
user_video_act_train_2_encoded_course_vec['predictions'],user_video_act_train_2_encoded_course_vec['predictions_prob'] =zip(*user_video_act_train_2_encoded_course_vec.progress_apply(lambda row: prediction_train_seperately(row['course_vecs_LR'],row['course_vecs_CNN']), axis=1))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7296/7296 [02:10<00:00, 55.92it/s]


In [31]:
user_video_act_train_2_encoded_course_vec['predictions'],user_video_act_train_2_encoded_course_vec['predictions_prob'] =zip(*user_video_act_train_2_encoded_course_vec.progress_apply(lambda row: prediction_train_jointly(row['course_vecs_LR'],row['course_vecs_CNN']), axis=1))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7296/7296 [00:39<00:00, 184.43it/s]


In [18]:
user_video_act_train_2_encoded_course_vec['predictions'],user_video_act_train_2_encoded_course_vec['predictions_prob'] =zip(*user_video_act_train_2_encoded_course_vec.progress_apply(lambda row: prediction_single_model(row['course_vecs_LR'],row['course_vecs_CNN']), axis=1))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7296/7296 [01:09<00:00, 104.45it/s]


In [31]:
final_result = user_video_act_train_2_encoded_course_vec[['predictions']].values.tolist()
final_result_prob = user_video_act_train_2_encoded_course_vec[['predictions_prob']].values.tolist()
ground_truth = user_video_act_train_2_encoded_course_vec[['label_list']].values.tolist()
# test = final_result.values.tolist()
final_result = [ item for elem in final_result for item in elem]
final_result = [ item for elem in final_result for item in elem]
final_result_prob = [ item for elem in final_result_prob for item in elem]
final_result_prob = [ item for elem in final_result_prob for item in elem]
ground_truth = [ item for elem in ground_truth for item in elem]
ground_truth = [ item for elem in ground_truth for item in elem]
acc = calculate_acc(final_result,ground_truth)
acc

0.8219608613450536

In [32]:
auc = roc_auc_score(ground_truth, final_result_prob)
auc

0.8914712172336592

In [33]:
f1=f1_score(ground_truth, final_result, average='macro')
f1

0.7693714799010429